# Hands-On ML.NET-RestaurantFeedback

In [8]:
// ML.NET Nuget packages installation
#r "nuget:Microsoft.ML" 

## Using C# Class

In [9]:
using Microsoft.ML;
using Microsoft.ML.Data;
using System;
using System.IO;

## Declare data-classes for input data and predictions

In [10]:
public class RestaurantFeedback
{
    [LoadColumn(0)]
    public bool Label { get; set; }

    [LoadColumn(1)]
    public string Text { get; set; }
}

public class RestaurantPrediction
{
    [ColumnName("PredictedLabel")]
    public bool Prediction { get; set; }

    public float Probability { get; set; }

    public float Score { get; set; }
}

In [11]:
public const string MODEL_FILENAME = "chapter2.mdl";

static readonly string trainingFileName = Path.Combine(Environment.CurrentDirectory, "./Datasets/RestaurantFeedback/sampledata.csv");

protected static string ModelPath => Path.Combine(AppContext.BaseDirectory, MODEL_FILENAME);

MLContext MlContext = new MLContext(2020);

## Treino

In [12]:
var trainingDataView = MlContext.Data.LoadFromTextFile<RestaurantFeedback>(trainingFileName);

var dataSplit = MlContext.Data.TrainTestSplit(trainingDataView, testFraction: 0.2);

var dataProcessPipeline = MlContext.Transforms.Text.FeaturizeText(
    outputColumnName: "Features", 
    inputColumnName: nameof(RestaurantFeedback.Text));

var sdcaRegressionTrainer = MlContext.BinaryClassification.Trainers.SdcaLogisticRegression(
    labelColumnName: nameof(RestaurantFeedback.Label), 
    featureColumnName: "Features");
var trainingPipeline = dataProcessPipeline.Append(sdcaRegressionTrainer);

ITransformer trainedModel = trainingPipeline.Fit(dataSplit.TrainSet);
MlContext.Model.Save(trainedModel, dataSplit.TrainSet.Schema, ModelPath);

var testSetTransform = trainedModel.Transform(dataSplit.TestSet);

var modelMetrics = MlContext.BinaryClassification.Evaluate(
    data: testSetTransform, 
    labelColumnName: nameof(RestaurantFeedback.Label), 
    scoreColumnName: nameof(RestaurantPrediction.Score));

Console.WriteLine($"Area Under Curve: {modelMetrics.AreaUnderRocCurve:P2}{Environment.NewLine}" +
                  $"Area Under Precision Recall Curve: {modelMetrics.AreaUnderPrecisionRecallCurve:P2}{Environment.NewLine}" +
                  $"Accuracy: {modelMetrics.Accuracy:P2}{Environment.NewLine}" +
                  $"F1Score: {modelMetrics.F1Score:P2}{Environment.NewLine}" +
                  $"Positive Recall: {modelMetrics.PositiveRecall:#.##}{Environment.NewLine}" +
                  $"Negative Recall: {modelMetrics.NegativeRecall:#.##}{Environment.NewLine}");

Area Under Curve: 0,00%
Area Under Precision Recall Curve: 41,67%
Accuracy: 33,33%
F1Score: 50,00%
Positive Recall: ,5
Negative Recall: 



In [13]:
DataViewSchema modelSchema;
ITransformer mlModel = MlContext.Model.Load(ModelPath, out modelSchema);

var predictionEngine = MlContext.Model.CreatePredictionEngine<RestaurantFeedback, RestaurantPrediction>(mlModel);

var prediction = predictionEngine.Predict(new RestaurantFeedback { Text =  "bad" });

In [7]:
 Console.WriteLine($"The feedback is predicted to be:{Environment.NewLine}{(prediction.Prediction ? "Negative" : "Positive")} at a {prediction.Probability:P0} confidence");

The feedback is predicted to be:
Positive at a 43% confidence
